In [ ]:
# reload during dev
from importlib import reload
import PET_raw_data
PET_raw_data = reload(PET_raw_data)


import PET_raw_data
dict_load = {
             '.\data\[Adt Income] adult.csv'      : {'describe_params' : {'missing_level' : {'data': {'?'}}}}
            ,'.\data\[Census] adult.data'         : {'read_params'     : {'header_exist' : 'N'
                                                                         ,'header'       : ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
                                                                         }
                                                    ,'describe_params' : {'missing_level' : {'data': {'?'}}}
                                                    }
            ,'.\data\[NHANES] B.csv'              : {'read_params'     : {'header_exist' : 'N'
                                                                         ,'header'       : ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                                                                         }
                                                    }
            ,'.\data\[Retail] Online Retail.xlsx' : {'read_method'     : 'pandas_xlsx'
                                                    ,'read_params'     : {'sheet_name' : 'Online Retail'
                                                                         ,'str_col'    : ['InvoiceNo','StockCode','InvoiceDate','CustomerID']
                                                                         }
                                                    }
            ,'.\data\[UK-US PF] va_household_ver_1_9_instance_2.csv' : {'read_params' : {'str_col' : ['hid','rlid','admin1','admin2']}}
            ,'.\data\[UK-US PF] va_person_ver_1_9_instance_2.csv'    : {'read_params' : {'str_col' : ['hid','pid','person_number','sex']}}
            }

__temp = {}
for filepath ,file_params in dict_load.items():
    file_params_init = {'read_params'     : {'label_encoding' : 'Y'}
                       ,'describe'        : 'N'
                       ,'describe_params' : {'correlation'   : 'Y'
                                            ,'collinearity'  : 'Y'}
                       ,'report'          : 'N'
                       ,'report_params'   : {'print_report' : 'N'
                                            ,'save_report'  : 'Y'}
                       }
    file_params_init.update(file_params)
    print(f"Now is {filepath}")
    __temp[filepath] = PET_raw_data.load_data(filepath ,file_params_init)
    break

In [ ]:
__temp['.\data\[Adt Income] adult.csv'].data_oricast.dtypes

In [ ]:
__temp['.\data\[Adt Income] adult.csv'].data_label_encoding.head(6)

In [28]:
# import time
# from datetime import datetime
# import pytz
# from sdv.metadata import SingleTableMetadata
# from sdv.single_table import GaussianCopulaSynthesizer ,CTGANSynthesizer ,TVAESynthesizer ,CopulaGANSynthesizer
n_sample = 10
library_name = 'SDV'



filename     = '[Adt Income] adult.csv'
out_filename = '[Adt Income]'



time_start = time.time()
__data         = __temp[f".\data\{filename}"].data.copy()
__data_row_num = __data.shape[0]
metadata       = SingleTableMetadata()
metadata.detect_from_dataframe(__data)
##
__data_oricast   = __temp[f".\data\{filename}"].data_oricast.copy()
metadata_oricast = SingleTableMetadata()
metadata_oricast.detect_from_dataframe(__data)
##
__data_label_encoding   = __temp[f".\data\{filename}"].data_label_encoding.copy()
metadata_label_encoding = SingleTableMetadata()
metadata_label_encoding.detect_from_dataframe(__data_label_encoding)
print(f"讀 2*metafile 時間 {round(time.time()-time_start ,4)} 秒")



# method_name = 'GaussianCoupula'
# method_name = 'CTGAN'
# method_name = 'TVAE'
method_name = 'CoupulaGAN'



print(f"現在套件是{method_name}")
time_start = time.time()
if   method_name == 'GaussianCoupula':
    synthesizer = GaussianCopulaSynthesizer(metadata)
    synthesizer.fit(__data)
elif method_name == 'CTGAN':
    synthesizer = CTGANSynthesizer(metadata)
    synthesizer.fit(__data_oricast) # _label_encoding)
elif method_name == 'TVAE':
    synthesizer = TVAESynthesizer(metadata)
    synthesizer.fit(__data_label_encoding)
elif method_name == 'CoupulaGAN':
    synthesizer = CopulaGANSynthesizer(metadata)
    synthesizer.fit(__data_label_encoding)
# else:
#     break
print(f"訓練時間 {round(time.time()-time_start ,4)} 秒")




time_start = time.time()
for i in range(n_sample):
    synthesizer.reset_sampling()
    synthetic_data = synthesizer.sample(num_rows   = __data_row_num
                                        ,batch_size = 1000
                                        )
print(f"合成 {n_sample} 次時間 {round(time.time()-time_start ,4)} 秒")



synthetic_data.head(6)

exectime = datetime.now().astimezone(pytz.timezone('Asia/Taipei'))
synthetic_data.to_csv(f".\data_dpsd\{out_filename}_[{library_name}]_[{method_name}]_{exectime.strftime('%Y%m%d_%H%M%S')}_{exectime.tzinfo.zone.replace('/' ,'_')}.csv")

訓練時間 7016.4251 秒


Sampling rows: : 49000it [00:17, 2729.33it/s]                         
Sampling rows: : 49000it [00:16, 2947.39it/s]                         
Sampling rows: : 49000it [00:17, 2783.27it/s]                         
Sampling rows: : 49000it [00:16, 2956.05it/s]                         
Sampling rows: : 49000it [00:17, 2788.52it/s]                         
Sampling rows: : 49000it [00:16, 2969.39it/s]                         
Sampling rows: : 49000it [00:16, 3009.53it/s]                         
Sampling rows: : 49000it [00:16, 2957.37it/s]                         
Sampling rows: : 49000it [00:18, 2685.68it/s]                         
Sampling rows: : 49000it [00:16, 2910.27it/s]                         


合成 10 次時間 170.8687 秒
